In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Set the Environment

In [ ]:
connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://13.57.193.25:9000/",        "nid": 3}}

env = connections["custom"] # mainnet, yeouido, euljiro, pagoda, custom

In [ ]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
import requests
import json
import pickle
import csv
import os
from datetime import datetime
ICX = 1000000000000000000 # 10**18

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [ ]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [ ]:
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

# Mainnet
# 

# Yeouido Testnet
# sICX_SCORE = "cxda7ee5bd3980530993a946ee544f67f59892ac06"
# bnUSD_SCORE = "cxb1bc4ba972d77f052d2d047a946dd2dd8ac36f38"
# BAL_SCORE = "cxbf1cd887fd661130cc16e62cfd149b0560900d1d"

# Tbears in Server
sICX_SCORE = ""
bnUSD_SCORE = ""
BAL_SCORE = ""
DEX = ""

ICX = 1000000000000000000

In [ ]:
# balanced_test wallet address = hx3f01840a599da07b0f620eeae7aa9c574169a4be
# wallet = KeyWallet.load("./keystores/balanced_test.json", "xxx")
wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")
deployer_wallet = wallet

contract = "core_contracts/dex.zip"
# contract = "token_contracts/sicx.zip"
# contract = "token_contracts/bnUSD.zip"
# contract = "token_contracts/bal.zip"

In [ ]:
transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(GALEN)\
    .value(1000000 * ICX)\
    .step_limit(1000000) \
    .nid(NID) \
    .nonce(2) \
    .version(3) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

In [ ]:
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contract))\
    .build()

# estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = 4000100000
signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
# sICX_SCORE = res['scoreAddress']
# bnUSD_SCORE = res['scoreAddress']
# BAL_SCORE = res['scoreAddress']

res

In [ ]:
res

In [ ]:
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(SAMP_SCORE)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contract))\
    .build()

#  estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = 4000100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

### Set Admin

In [ ]:
params = {"_admin": "hx3f01840a599da07b0f620eeae7aa9c574169a4be"}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(sICX_SCORE)\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setAdmin")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
get_tx_result(tx_hash)

### Mint tokens

In [ ]:
params = {"_account": GALEN, "_amount": "10000000000000000000000"}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(sICX_SCORE)\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("mintTo")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
get_tx_result(tx_hash)

In [ ]:
params = {"account": 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'}
call = CallBuilder().from_(wallet.get_address())\
                    .to(BAL_SCORE)\
                    .method("balanceOf")\
                    .params(params)\
                    .build()
result = icon_service.call(call)